# 📊 Análise Exploratória de Dados - Imóveis Rurais

**Autor:** Marcos Paulo Roriz Lima Reis  
**RA:** 22007534  
**Email:** marcos.paulor@sempreceub.com  
**Curso:** Engenharia da Computação - UniCEUB  

## 🎯 Objetivos
- Analisar dados de imóveis rurais coletados
- Identificar padrões e correlações
- Detectar e tratar outliers
- Preparar dados para modelagem

In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuração dos gráficos
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# Carregamento dos dados
data_path = Path('../data/imoveis_rurais.csv')
df = pd.read_csv(data_path)

print(f"📊 Dataset carregado com {len(df)} registros e {len(df.columns)} colunas")
print(f"📏 Dimensões: {df.shape}")
df.head()

In [ ]:
# Informações gerais do dataset
print("📋 Informações do Dataset:")
print("=" * 40)
df.info()

print("\n📊 Estatísticas Descritivas:")
print("=" * 40)
df.describe()

In [ ]:
# Verificação de valores ausentes
print("🔍 Valores Ausentes:")
print("=" * 40)
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Valores Ausentes': missing_values,
    'Percentual (%)': missing_percent
})

missing_df = missing_df[missing_df['Valores Ausentes'] > 0].sort_values('Valores Ausentes', ascending=False)
print(missing_df)

In [ ]:
# Visualização da distribuição de preços
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Histograma de preços
axes[0,0].hist(df['preco'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('📊 Distribuição de Preços')
axes[0,0].set_xlabel('Preço (R$)')
axes[0,0].set_ylabel('Frequência')

# Boxplot de preços
axes[0,1].boxplot(df['preco'])
axes[0,1].set_title('📦 Boxplot de Preços')
axes[0,1].set_ylabel('Preço (R$)')

# Histograma de área
axes[1,0].hist(df['area_ha'], bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1,0].set_title('📊 Distribuição de Áreas')
axes[1,0].set_xlabel('Área (hectares)')
axes[1,0].set_ylabel('Frequência')

# Boxplot de área
axes[1,1].boxplot(df['area_ha'])
axes[1,1].set_title('📦 Boxplot de Áreas')
axes[1,1].set_ylabel('Área (hectares)')

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot: Preço vs Área
plt.figure(figsize=(12, 8))
plt.scatter(df['area_ha'], df['preco'], alpha=0.6, color='coral')
plt.title('🏠 Relação entre Área e Preço dos Imóveis Rurais')
plt.xlabel('Área (hectares)')
plt.ylabel('Preço (R$)')
plt.grid(True, alpha=0.3)

# Linha de tendência
z = np.polyfit(df['area_ha'], df['preco'], 1)
p = np.poly1d(z)
plt.plot(df['area_ha'], p(df['area_ha']), "r--", alpha=0.8, label='Linha de Tendência')
plt.legend()
plt.show()

# Correlação
correlacao = df['area_ha'].corr(df['preco'])
print(f"📈 Correlação Área vs Preço: {correlacao:.3f}")

In [ ]:
# Matriz de correlação
# Selecionar apenas colunas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='coolwarm', 
            center=0,
            square=True,
            fmt='.2f')
plt.title('🔥 Matriz de Correlação - Variáveis Numéricas')
plt.tight_layout()
plt.show()

print("\n🔍 Correlações mais fortes com preço:")
price_corr = correlation_matrix['preco'].sort_values(ascending=False)
for var, corr in price_corr.items():
    if var != 'preco':
        print(f"  {var}: {corr:.3f}")

In [ ]:
# Análise de outliers usando IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Detectar outliers em preço
price_outliers, price_lower, price_upper = detect_outliers_iqr(df, 'preco')
print(f"💰 Outliers em Preço: {len(price_outliers)} registros")
print(f"   Limites: R$ {price_lower:,.2f} - R$ {price_upper:,.2f}")

# Detectar outliers em área
area_outliers, area_lower, area_upper = detect_outliers_iqr(df, 'area_ha')
print(f"🏞️ Outliers em Área: {len(area_outliers)} registros")
print(f"   Limites: {area_lower:.2f} - {area_upper:.2f} hectares")

# Mostrar alguns exemplos de outliers
if len(price_outliers) > 0:
    print("\n💸 Exemplos de outliers em preço:")
    print(price_outliers[['preco', 'area_ha', 'cidade']].head())

In [ ]:
# Limpeza de dados - remover outliers extremos
print("🧹 Iniciando limpeza de dados...")
print(f"   Dataset original: {len(df)} registros")

# Remover outliers extremos (percentis 5% e 95%)
price_p5 = df['preco'].quantile(0.05)
price_p95 = df['preco'].quantile(0.95)
area_p5 = df['area_ha'].quantile(0.05)
area_p95 = df['area_ha'].quantile(0.95)

df_clean = df[
    (df['preco'] >= price_p5) & (df['preco'] <= price_p95) &
    (df['area_ha'] >= area_p5) & (df['area_ha'] <= area_p95)
].copy()

print(f"   Dataset limpo: {len(df_clean)} registros")
print(f"   Registros removidos: {len(df) - len(df_clean)}")
print(f"   Percentual mantido: {len(df_clean)/len(df)*100:.1f}%")

In [ ]:
# Comparação antes e depois da limpeza
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Preço - antes
axes[0,0].hist(df['preco'], bins=30, alpha=0.7, color='red', edgecolor='black')
axes[0,0].set_title('📊 Preços - Antes da Limpeza')
axes[0,0].set_xlabel('Preço (R$)')

# Preço - depois
axes[0,1].hist(df_clean['preco'], bins=30, alpha=0.7, color='green', edgecolor='black')
axes[0,1].set_title('✅ Preços - Após Limpeza')
axes[0,1].set_xlabel('Preço (R$)')

# Área - antes
axes[1,0].hist(df['area_ha'], bins=30, alpha=0.7, color='red', edgecolor='black')
axes[1,0].set_title('📊 Área - Antes da Limpeza')
axes[1,0].set_xlabel('Área (hectares)')

# Área - depois
axes[1,1].hist(df_clean['area_ha'], bins=30, alpha=0.7, color='green', edgecolor='black')
axes[1,1].set_title('✅ Área - Após Limpeza')
axes[1,1].set_xlabel('Área (hectares)')

plt.tight_layout()
plt.show()

In [ ]:
# Estatísticas finais
print("📈 Estatísticas Finais - Dataset Limpo:")
print("=" * 50)
print(f"📊 Registros: {len(df_clean)}")
print(f"💰 Preço médio: R$ {df_clean['preco'].mean():,.2f}")
print(f"💰 Preço mediano: R$ {df_clean['preco'].median():,.2f}")
print(f"🏞️ Área média: {df_clean['area_ha'].mean():.2f} hectares")
print(f"🏞️ Área mediana: {df_clean['area_ha'].median():.2f} hectares")
print(f"📈 Correlação Área vs Preço: {df_clean['area_ha'].corr(df_clean['preco']):.3f}")

# Salvar dataset limpo
output_path = Path('../data/imoveis_rurais_tratados.csv')
df_clean.to_csv(output_path, index=False)
print(f"\n💾 Dataset limpo salvo em: {output_path}")
print("✅ Análise exploratória concluída!")